In [1]:
!pip install transformers torch PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.1 MB/s eta 0:00:00


Installs all necessary libraries to run transformer-based GenAI models and extract text from PDF files inside Google Colab.

In [2]:
from transformers import pipeline
import PyPDF2


Imports the Hugging Face pipeline API for loading GenAI models and PyPDF2 for reading and extracting text from PDF documents.

In [3]:
qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad"
)

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn"
)

ner_pipeline = pipeline(
    "ner",
    grouped_entities=True
)

print("All GenAI pipelines loaded successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


All GenAI pipelines loaded successfully


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


Initializes three transformer-based GenAI pipelines for question answering, text summarization, and named entity recognition using pretrained models.

In [4]:
from google.colab import files

uploaded = files.upload()

pdf_filename = list(uploaded.keys())[0]
print("Uploaded file:", pdf_filename)


Saving GenAI_Unit1.pdf to GenAI_Unit1.pdf
Uploaded file: GenAI_Unit1.pdf


Allows the user to upload a textbook PDF file into the Colab environment and stores the uploaded file name for further processing.(I have uploaded pesu slides of genAI unit 1)

In [5]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
    return text

context_text = extract_text_from_pdf(pdf_filename)

print("Total characters extracted:", len(context_text))


Total characters extracted: 137392


Reads the uploaded PDF page by page and extracts all readable text into a single string to be used as the context for GenAI processing.

In [6]:
def chunk_text(text, chunk_size=400):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks

context_chunks = chunk_text(context_text)
print("Total chunks:", len(context_chunks))


Total chunks: 50


Splits the long textbook text into smaller chunks so that the transformer model can process large documents efficiently without token limits.

In [7]:
def ask_question(question, chunks):
    best_answer = ""
    best_score = 0

    for chunk in chunks:
        try:
            result = qa_pipeline(
                question=question,
                context=chunk
            )
            if result["score"] > best_score:
                best_score = result["score"]
                best_answer = result["answer"]
        except:
            continue

    return best_answer, best_score


Searches across all text chunks using an extractive QA model and returns the most confident answer found in the document.

In [8]:
question = "What is Machine Learning?"

answer, score = ask_question(question, context_chunks)

print("Question:", question)
print("Answer:", answer)
print("Confidence Score:", score)


Question: What is Machine Learning?
Answer: Large Language Models
Confidence Score: 0.8086193203926086


Demonstrates how a user can ask a natural language question and receive an answer extracted directly from the uploaded textbook.

In [9]:
summary = summarizer(
    context_text[:2000],
    max_length=200,
    min_length=80,
    do_sample=False
)

print("Chapter Summary:\n")
print(summary[0]["summary_text"])


Chapter Summary:

The slides are prepared from  various resources from the Universities from abroad and India. Some material is taken from reliable resources from internet throughout this course. Course Instructor: Dr. Arti Arya, CSE, PES University, Bangalore.Unit 1: NLP Basics and Word Embeddings. Unit 2: Generative AI and Its Applications. Unit 3: Transformer models.


Uses a generative transformer model to produce a concise summary of the textbook chapter, highlighting key concepts.

In [10]:
entities = ner_pipeline(context_text[:1500])

print("Key Entities Found:\n")
for ent in entities:
    print(ent["entity_group"], ":", ent["word"])


Key Entities Found:

PER : Arti Arya
ORG : CSE
ORG : PES University
LOC : Bangalore
MISC : AI
MISC : AI
LOC : India
PER : Sai Yashwanth
PER : Pooja Agarwal
PER : Arti Arya
MISC : AI
MISC : ##AI
MISC : ##AI
MISC : ##AI
MISC : GenA
MISC : ##A
MISC : AI
MISC : ChatGP
MISC : Gene
MISC : AI


Identifies important entities such as concepts, organizations, and names from the textbook content to help users quickly locate key terms.